In [58]:
# Import relevant libraries
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

"""
Requirements:
torch==1.4.0
transformers==2.9.0
pytorch_lightning==0.7.5

Model 1:
    1. Created an environment "t5_env" with following spec: torch==1.4.0,transformers==2.9.0,pytorch_lightning==0.7.5
    2. https://towardsdatascience.com/paraphrase-any-question-with-t5-text-to-text-transfer-transformer-pretrained-model-and-cbb9e35f1555
    3. conda activate t5_env
    4. python train.py
"""

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# https://huggingface.co/welcome <-- Your model will then be accessible through its identifier: username/model_name
"""
Anyone can load it from code:
    tokenizer = AutoTokenizer.from_pretrained("username/model_name")
    model = AutoModel.from_pretrained("username/model_name")
"""

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)


def paraphrase(sentence):
    text =  "paraphrase: " + sentence + " </s>"
    max_len = 256
    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 10
    beam_outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_masks,
            do_sample=True,
            max_length=256,
            top_k=120,
            top_p=0.98,
            early_stopping=True,
            num_return_sequences=10
        )
    print ("\nOriginal Question :: ",sentence)
    print ("Paraphrased Questions :: ")
    final_outputs =[]
    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
        if sent.lower() != sentence.lower() and sent not in final_outputs:
            final_outputs.append(sent)

    for i, final_output in enumerate(final_outputs):
        print("{}: {}".format(i, final_output))
        print("\n")
        # we will return the top result but for future this code can be modified to provide top n results
        return final_output

# Here we are not training any new model just using a pretrained model which is trained on Quora dataset and it gives
# already good results as the quora dataset is very huge  can be found in folder quora_dataset
import pandas as pd
file =('/content/eval (1).csv') #file for evaulating/testing
df = pd.read_csv(file,header=None)
df[1] = df.apply (lambda row: paraphrase(row[0]), axis=1)
df.to_csv('file_output.csv')

device  cpu


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should s


Original Question ::  question1
Paraphrased Questions :: 
0: Why did you ask you what you thought of the question1?



Original Question ::  where are the harvard medical, dental and school of public health located?
Paraphrased Questions :: 
0: Where are Yale medical schools located?



Original Question ::  which logo had the dw tardis insignia removed?
Paraphrased Questions :: 
0: Which logo was removed from Dw tardis Insignia?



Original Question ::  who did genghis khan unite before he began conquering the rest of eurasia?
Paraphrased Questions :: 
0: Who started genghis Khan and where did he begin?



Original Question ::  clergy are members of what group rather than of any local congregation?
Paraphrased Questions :: 
0: To what group are the clergy, rather than any churches?



Original Question ::  what does computational complexity theory most specifically seek to answer? 
Paraphrased Questions :: 
0: What questions do computational complexity theoretical engineers often ask

In [59]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Load evaluation dataset
df = pd.read_csv('/content/eval (1).csv', header=None)

# Assuming column 0 has original sentences and column 1 has paraphrased outputs
original_sentences = df[0].tolist()
paraphrased_sentences = df[1].tolist()

# Compute BLEU scores
bleu_scores = []
for original, paraphrased in zip(original_sentences, paraphrased_sentences):
    reference = [original.split()]
    candidate = paraphrased.split()
    score = sentence_bleu(reference, candidate)
    bleu_scores.append(score)

# Store results
df['BLEU Score'] = bleu_scores
df.to_csv('bleu_scores_output.csv', index=False)

# Print average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f'Average BLEU Score: {average_bleu}')

Average BLEU Score: 0.6562331438148147


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
